In [4]:
import optuna
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

In [13]:
class GalaxyNet(nn.Module):
    def __init__(self, n_conv_layers, num_filters,num_neurons1, num_neurons2):
        super().__init__()
        ### Maxpooling layer!!!!
        self.rgb = 1
        self.input_size = 324
        self.num_labels = 37
        self.loss_dict = { 'batch' : [], 'epoch' : [], 'vbatch' : [], 'vepoch' : []}


        stride = 2
        kernel_size = 3
        kernel_size_pool = 2
        ## convolutional layers
        self.convs = nn.ModuleList([nn.Conv2d(self.rgb, num_filters[0], kernel_size=kernel_size)])
        self.convs.append(nn.BatchNorm2d(num_filters[0]))
        output_size = (self.input_size - kernel_size + stride) // (stride*kernel_size_pool)
        for i in range(1,n_conv_layers):
            self.convs.append(nn.Conv2d(num_filters[i-1], num_filters[i], kernel_size=kernel_size)) # num filters are the number of channel the conv layer otputs.
            self.convs.append(nn.BatchNorm2d(num_filters[i]))
            output_size = (output_size - kernel_size + stride) // (stride*kernel_size_pool) #padding 0, dilation = 1
        self.pool = nn.MaxPool2d(kernel_size=kernel_size_pool)
        #self.convs.append(nn.dropout(p= value)) ## to be added in the future to test claims of BatchnOrm paper
        self.out_feature = num_filters[-1] *output_size * output_size # output size of the last conv layer
        self.fc1 = nn.Linear(self.out_feature, num_neurons1) # fully connected layer
        # dropout here if you want
        self.fc2 = nn.Linear(num_neurons1, num_neurons2)
        #dropout here if u want
        self.fc3 = nn.Linear(num_neurons2, self.num_labels) 

    def init_weights(self,  mode, n_conv_layers):
        if mode == 'relu': # perchè kaiming normal e non uniform??1
            for i in range(0, n_conv_layers*2, 2):
                nn.init.kaiming_normal_(self.convs[i].weight, nonlinearity=mode)
                if self.convs[i].bias is not None:
                    nn.init.constant_(self.convs[i].bias,0)
            nn.init.kaiming_normal_(self.fc1.weight, nonlinearity=mode)
            if self.fc1.bias is not None:
                nn.init.constant_(self.fc1.bias,0)
            nn.init.kaiming_normal_(self.fc2.weight, nonlinearity=mode)
            if self.fc2.bias is not None:
                nn.init.constant_(self.fc2.bias,0)
        elif mode == 'leaky_relu':
            for i in range(0, n_conv_layers*2, 2):
                nn.init.kaiming_normal_(self.convs[i].weight, a = 0.01, nonlinearity=mode)
                if self.convs[i].bias is not None:
                    nn.init.constant_(self.convs[i].bias,0)
            nn.init.kaiming_normal_(self.fc1.weight, nonlinearity=mode)
            if self.fc1.bias is not None:
                nn.init.constant_(self.fc1.bias,0)
            nn.init.kaiming_normal_(self.fc2.weight, nonlinearity=mode)
            if self.fc2.bias is not None:
                nn.init.constant_(self.fc2.bias,0) 
        nn.init.xavier_uniform_(self.fc3.weight)
        if self.fc3.bias is not None:
            nn.init.constant_(self.fc3.bias,0)
        return print('weights initialized with {}'.format(mode))        
        

    def forward(self,x, FUNZIONEDIATTIVAZIONE):
        for i in range(0, len(self.convs),2):
            x = self.pool(FUNZIONEDIATTIVAZIONE(self.convs[i](x)))
            x = self.convs[i+1](x) # batch norm layer
        x = torch.flatten(x,1) # flatten operation -> 1 dimensional
        x = FUNZIONEDIATTIVAZIONE(self.fc1(x)) # apply relu al'output dei fully connected
        x = FUNZIONEDIATTIVAZIONE(self.fc2(x)) # idem sopra
        x = self.fc3(x) # output di fc3, 37 neuroni -> 37 classi ideally 
        ## MAPPING ALSO HERE
        return x
    
    def set_rgb(self,bool):
        if bool:
            self.rgb = 3
        else:
            self.rgb = 1
        return self.rgb
    
    def log_the_loss(self, item,epoch=False): # per avere una history della loss???
        train = self.__getstate__()['training']
        print(train)
        if epoch and train:
            self.loss_dict['epoch'].append(item) ### get state of the model so you can ditch the validation parameter
        elif not epoch and train:
            self.loss_dict['batch'].append(item)
        elif not train and epoch:
            self.loss_dict['vepoch'].append(item)
        elif not train and not epoch:
            self.loss_dict['vbatch'].append(item)
        return item

            

In [14]:
#n_conv_layers, num_filters,num_neurons1, num_neurons2):

prova = GalaxyNet(2,[3,6,9],50, 45)

In [16]:
prova.set_rgb(True)

3

In [3]:
tensor = torch.randn(10,2,300,300)
m = nn.MaxPool2d(kernel_size=2)
n = nn.Conv2d(10,20,3)

x = m(n(tensor))

RuntimeError: Given groups=1, weight of size [20, 10, 3, 3], expected input[10, 2, 300, 300] to have 10 channels, but got 2 channels instead

In [2]:
torch.randn?

Docstring:
randn(*size, *, generator=None, out=None, dtype=None, layout=torch.strided, device=None, requires_grad=False, pin_memory=False) -> Tensor


Returns a tensor filled with random numbers from a normal distribution
with mean `0` and variance `1` (also called the standard normal
distribution).

.. math::
    \text{out}_{i} \sim \mathcal{N}(0, 1)

For complex dtypes, the tensor is i.i.d. sampled from a `complex normal distribution`_ with zero mean and
unit variance as

.. math::
    \text{out}_{i} \sim \mathcal{CN}(0, 1)

This is equivalent to separately sampling the real :math:`(\operatorname{Re})` and imaginary
:math:`(\operatorname{Im})` part of :math:`\text{out}_i` as

.. math::
    \operatorname{Re}(\text{out}_{i}) \sim \mathcal{N}(0, \frac{1}{2}),\quad
    \operatorname{Im}(\text{out}_{i}) \sim \mathcal{N}(0, \frac{1}{2})

The shape of the tensor is defined by the variable argument :attr:`size`.


Args:
    size (int...): a sequence of integers defining the shape of the outp